# Theme of this notebook - how to calculate performance metrics (CV, Sharpe Ratio, Treynor Ratio, Sortino Ratio)
* These are codes for calculating performance metrics of a portfolio
* Let's practice throughout solving sample questions.

# Question
Using monthly data for the period 2018M1 to 2022M12, compute and compare the following finance ratios for APPL and TSLA:<br>
1. Coefficient of Variation (CV)
2. Sharpe Ratio
3. Treynor Ratio
4. Sharpe Ratio

# 1.Coefficient of Variation (CV)

Basically, I followed the steps below.<br>
1. Choose one of the models (*I adopted one-factor model).<br>
2. Find E($R_a$) = $\hat{\alpha}$ +$\hat{\beta}R_m$ for the time period.<br>
3. Calculate excess returns, its volatility, beta, negative returns, etc.

In [1]:
# First, retrive data from Yahoo Finance
# Before that, let's install pandas-datareader
! pip install pandas-datareader

In [2]:
# Import libraries
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import matplotlib.pyplot as plt
import statsmodels.api as sm
import yfinance as yf

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [3]:
# Retrive Apple's data from 2018M1 to 2022M12 on a monthly basis
apple = yf.download('AAPL', start='2018-01-01', end='2022-12-31', interval='1mo')
# Retrive Tesla (TSLA)'s data from 2018M1 to 2022M12 on a monthly basis
tesla = yf.download('TSLA',start='2018-01-01', end='2022-12-31', interval='1mo')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [4]:
# Set DataFrames
apple = pd.DataFrame(apple.loc['2018-01-01':'2022-12-31', 'Adj Close'])
tesla = pd.DataFrame(tesla.loc['2018-01-01':'2022-12-31', 'Adj Close'])
returns = pd.DataFrame()

<ul>
  <li>Note, the <b>logarithmic return</b> of a security can be defined as follows $$r_{i,t} = ln(\frac{P_{i,t}}{P_{i,t-1}}),$$ where $P_{i,t}$ is the price of security $i$ at time $t$.</li>
</ul>

In [5]:
# Calculate each return
returns['AAPL_ret'] = np.log(apple['Adj Close'] / apple['Adj Close'].shift(1))
returns['TSLA_ret'] = np.log(tesla['Adj Close'] / tesla['Adj Close'].shift(1))

returns

,AAPL_ret,TSLA_ret
Date,,
2018-01-01,NaN,NaN
2018-02-01,0.061892,-0.032267
2018-03-01,-0.055735,-0.253920
2018-04-01,-0.015134,0.099255
2018-05-01,0.122893,-0.031698
2018-06-01,-0.005614,0.186043
2018-07-01,0.027599,-0.140021
2018-08-01,0.179172,0.011737
2018-09-01,-0.004837,-0.130439


In [6]:
# Calculate mean and volatility(= standard deviation)
mean_std = pd.DataFrame({'Mean': returns.dropna().mean(),
                        'Volatility': returns.dropna().std()})

mean_std

,Mean,Volatility
AAPL_ret,0.020052,0.092818
TSLA_ret,0.027992,0.197279


Let's calculate Coefficient of Variation (CV)!

# <b>Coefficient of Variation (CV) = $\frac{Standard \ Deviation(s)}{Sample \ Mean (\bar{x})}$ </b>

In [7]:
#Calculate CV
cv_aapl = mean_std.iloc[0,1]/mean_std.iloc[0,0]
cv_tsla = mean_std.iloc[1,1]/mean_std.iloc[1,0]
print(f"Coefficient  of Variation of Apple = {cv_aapl}")
print(f"Coefficient  of Variation of Tesla = {cv_tsla}")

Coefficient  of Variation of Apple = 4.628854798212706
Coefficient  of Variation of Tesla = 7.04770348889829


#2. Sharpe Ratio

* In order to calculate Sharpe Ratio, I adopted ten-year treasury (^TNX) as a risk-free rate.

In [14]:
# Retrive 10-year treasury data from 2018M1 to 2022M12 on a monthly basis
risk_free = yf.download('^TNX', start='2018-01-01', end='2022-12-31', interval='1mo')

[*********************100%%**********************]  1 of 1 completed


In [15]:
# Set monthly returns of 10-year treasury
# Data of 10-year treasury are basically displayed percentage
# To convert annual rate of return into monthly rate of return, divide returns by 1200(100 * 12-month)
returns['risk_free_ret'] = (risk_free.loc['2018-01-01':'2022-12-31', 'Adj Close']) / 1200
returns

,AAPL_ret,TSLA_ret,risk_free_ret,Excess_return_AAPL,Excess_return_TSLA
Date,,,,,
2018-01-01,NaN,NaN,0.002267,NaN,NaN
2018-02-01,0.061892,-0.032267,0.002390,0.059502,-0.034657
2018-03-01,-0.055735,-0.253920,0.002284,-0.058019,-0.256205
2018-04-01,-0.015134,0.099255,0.002447,-0.017580,0.096808
2018-05-01,0.122893,-0.031698,0.002352,0.120542,-0.034050
2018-06-01,-0.005614,0.186043,0.002374,-0.007989,0.183669
2018-07-01,0.027599,-0.140021,0.002470,0.025129,-0.142491
2018-08-01,0.179172,0.011737,0.002377,0.176795,0.009360
2018-09-01,-0.004837,-0.130439,0.002547,-0.007383,-0.132986


In [16]:
# Calculate excess returns of AAPL and TSLA
# Excess return = Return on each security - Risk free rate
returns['Excess_return_AAPL'] = (returns.iloc[:,0]-returns.iloc[:,2])
returns['Excess_return_TSLA'] = (returns.iloc[:,1]-returns.iloc[:,2])
returns

,AAPL_ret,TSLA_ret,risk_free_ret,Excess_return_AAPL,Excess_return_TSLA
Date,,,,,
2018-01-01,NaN,NaN,0.002267,NaN,NaN
2018-02-01,0.061892,-0.032267,0.002390,0.059502,-0.034657
2018-03-01,-0.055735,-0.253920,0.002284,-0.058019,-0.256205
2018-04-01,-0.015134,0.099255,0.002447,-0.017580,0.096808
2018-05-01,0.122893,-0.031698,0.002352,0.120542,-0.034050
2018-06-01,-0.005614,0.186043,0.002374,-0.007989,0.183669
2018-07-01,0.027599,-0.140021,0.002470,0.025129,-0.142491
2018-08-01,0.179172,0.011737,0.002377,0.176795,0.009360
2018-09-01,-0.004837,-0.130439,0.002547,-0.007383,-0.132986


In [17]:
# Create table to calculate the standard deviation of excess return
mean_std = pd.DataFrame({'Mean': returns.dropna().mean(),
                        'Volatility': returns.dropna().std()})

mean_std

,Mean,Volatility
AAPL_ret,0.020052,0.092818
TSLA_ret,0.027992,0.197279
risk_free_ret,0.001697,0.000780
Excess_return_AAPL,0.018355,0.093048
Excess_return_TSLA,0.026295,0.197623


Let's calculate Sharpe ratio!

# <b>Sharpe Ratio = $\frac{R_p - Rf}{\sigma_p}$</b><br>
$R_p$ = Return of portfolio<br>
$R_f$ = Risk-free rate<br>
$\sigma_p$ = Standard Deviation of the portfolio's excess return<br>

In [18]:
#Calculate Sharpe Ratio
sr_aapl = mean_std.iloc[3,0] / mean_std.iloc[3,1]
sr_tsla = mean_std.iloc[4,0] / mean_std.iloc[4,1]
print(f"Sharpe Ratio of Apple = {sr_aapl}")
print(f"Sharpe Ratio of Tesla = {sr_tsla}")

Sharpe Ratio of Apple = 0.1972608773237108
Sharpe Ratio of Tesla = 0.1330544858294973


#3. Treinor Ratio

* In order to calculate Treynor Ratio, I adopeted S&P500 (^GSPC) as a benchmark.

In [19]:
# Retrive apple's data from 2018M1 to 2022M12 on a monthly basis
apple = yf.download('AAPL', start='2018-01-01', end='2022-12-31', interval='1mo')
# Retrive tesla's data from 2018M1 to 2022M12 on a monthly basis
tesla = yf.download('TSLA', start='2018-01-01', end='2022-12-31', interval='1mo')
# Retrive S&P 500(^GSPC)'s data from 2018M1 to 2022M12 on a monthly basis
sp500 = yf.download('^GSPC',start='2018-01-01', end='2022-12-31', interval='1mo')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [20]:
# Calculate logarithmic returns
apple = pd.DataFrame(apple.loc['2018-01-01':'2022-12-31', 'Adj Close'])
tesla = pd.DataFrame(tesla.loc['2018-01-01':'2022-12-31', 'Adj Close'])
sp500 = pd.DataFrame(sp500.loc['2018-01-01':'2022-12-31', 'Adj Close'])
apple['AAPL_ret'] = np.log(apple['Adj Close'] / apple['Adj Close'].shift(1))
tesla['TSLA_ret'] = np.log(tesla['Adj Close'] / tesla['Adj Close'].shift(1))
sp500['S&P500_ret'] = np.log(sp500['Adj Close'] / sp500['Adj Close'].shift(1))

# Reset Variable
apple = apple['AAPL_ret']
tesla = tesla['TSLA_ret']
sp500 = sp500['S&P500_ret']

# Set intercept to forecast beta by using statsmodel
sp500 = sm.add_constant(sp500)

In [21]:
# Forecast α and β of Apple by using statsmodel
model_apple = sm.OLS(apple.dropna(),sp500.dropna())
result_apple = model_apple.fit()
result_apple.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               AAPL_ret   R-squared:                       0.530
Model:                            OLS   Adj. R-squared:                  0.522
Method:                 Least Squares   F-statistic:                     64.37
Date:                Sat, 27 Jan 2024   Prob (F-statistic):           6.29e-11
Time:                        09:16:22   Log-Likelihood:                 79.333
No. Observations:                  59   AIC:                            -154.7
Df Residuals:                      57   BIC:                            -150.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0136      0.008      1.617      0.111      -0.003       0.030
S&P500_ret     1.2451      0.155      8.023      0.000       0.934       1.556
==============================================================================
Omnibus:                       11.875   Durbin-Watson:                   1.937
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               15.783
Skew:                          -0.738   Prob(JB):                     0.000374
Kurtosis:                       5.060   Cond. No.                         18.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [22]:
# Set Apple's beta
apple_beta = result_apple.params[1]
apple_beta

1.245145144971388

In [23]:
# Forecast α and β of Tesla by using statsmodel
model_tesla = sm.OLS(tesla.dropna(),sp500.dropna())
result_tesla = model_tesla.fit()
result_tesla.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               TSLA_ret   R-squared:                       0.255
Model:                            OLS   Adj. R-squared:                  0.242
Method:                 Least Squares   F-statistic:                     19.55
Date:                Sat, 27 Jan 2024   Prob (F-statistic):           4.46e-05
Time:                        09:16:30   Log-Likelihood:                 21.251
No. Observations:                  59   AIC:                            -38.50
Df Residuals:                      57   BIC:                            -34.35
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0184      0.022      0.820      0.415      -0.027       0.063
S&P500_ret     1.8364      0.415      4.421      0.000       1.005       2.668
==============================================================================
Omnibus:                        0.819   Durbin-Watson:                   1.519
Prob(Omnibus):                  0.664   Jarque-Bera (JB):                0.597
Skew:                           0.246   Prob(JB):                        0.742
Kurtosis:                       2.967   Cond. No.                         18.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [24]:
# Set Tesla's beta
tesla_beta = result_tesla.params[1]
tesla_beta

1.8364387280617502

Let's calculate Treynor Ratio!

# <b>Treynor Ratio = $\frac{R_p - Rf}{\beta_p}$</b><br>
$R_p$ = Return of portfolio<br>
$R_f$ = Risk-free rate<br>
$\sigma_p$ = Beta of the portfolion<br>

In [25]:
mean_std

,Mean,Volatility
AAPL_ret,0.020052,0.092818
TSLA_ret,0.027992,0.197279
risk_free_ret,0.001697,0.000780
Excess_return_AAPL,0.018355,0.093048
Excess_return_TSLA,0.026295,0.197623


In [26]:
# Calculate Treynor Ratio
tr_aapl = mean_std.iloc[3,0] / apple_beta
tr_tsla = mean_std.iloc[4,0] / tesla_beta
print(f"Treynor Ratio of Apple = {tr_aapl}")
print(f"Treynor Ratio of Tesla = {tr_tsla}")

Treynor Ratio of Apple = 0.01474106824751307
Treynor Ratio of Tesla = 0.014318286312315517


#4. Sortino Ratio

In [28]:
# Retrive data from 2018M1 to 2022M12 on a monthly basis
apple = yf.download('AAPL', start='2018-01-01', end='2022-12-31', interval='1mo')
tesla = yf.download('TSLA', start='2018-01-01', end='2022-12-31', interval='1mo')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [29]:
# Let's extract negative returns of AAPL and Tesla
apple_returns = pd.DataFrame()
apple_returns['AAPL_ret'] = np.log(apple['Adj Close'] / apple['Adj Close'].shift(1))
negative_return_aapl = apple_returns[apple_returns['AAPL_ret'] <= 0]
negative_return_aapl

,AAPL_ret
Date,
2018-03-01,-0.055735
2018-04-01,-0.015134
2018-06-01,-0.005614
2018-09-01,-0.004837
2018-10-01,-0.030952
2018-11-01,-0.203396
2018-12-01,-0.120605
2019-05-01,-0.136476
2019-08-01,-0.020391


In [30]:
tesla_returns = pd.DataFrame()
tesla_returns['TSLA_ret'] = np.log(tesla['Adj Close'] / tesla['Adj Close'].shift(1))
negative_return_tsla = tesla_returns[tesla_returns['TSLA_ret'] <= 0]
negative_return_tsla

,TSLA_ret
Date,
2018-02-01,-0.032267
2018-03-01,-0.253920
2018-05-01,-0.031698
2018-07-01,-0.140021
2018-09-01,-0.130439
2018-12-01,-0.051762
2019-01-01,-0.080629
2019-03-01,-0.133656
2019-04-01,-0.159124


In [32]:
# Calculate negative returns std
negative_returns_std = pd.DataFrame({'AAPL': negative_return_aapl.std(),
                                    'TSLA': negative_return_tsla.std()})

negative_returns_std

,AAPL,TSLA
AAPL_ret,0.050257,NaN
TSLA_ret,NaN,0.091252


Let's calculate Sortino Ratio!

# <b>Sortino Ratio = $\frac{R_p - Rf}{\sigma_d}$</b><br>
$R_p$ = Return of portfolio<br>
$R_f$ = Risk-free rate<br>
$\sigma_d$ = Standard Deviation of Negative Returns on the portfolio (downside)<br>

In [33]:
mean_std

,Mean,Volatility
AAPL_ret,0.020052,0.092818
TSLA_ret,0.027992,0.197279
risk_free_ret,0.001697,0.000780
Excess_return_AAPL,0.018355,0.093048
Excess_return_TSLA,0.026295,0.197623


In [34]:
sor_aapl = mean_std.iloc[3,0] / negative_returns_std.iloc[0,0]
sor_tsla = mean_std.iloc[4,0] / negative_returns_std.iloc[1,1]
print(f"Sortino Ratio of Apple = {sor_aapl}")
print(f"Sortino Ratio of Tesla = {sor_tsla}")

Sortino Ratio of Apple = 0.3652150606086473
Sortino Ratio of Tesla = 0.28815292077836796


# Merge the results

In [35]:
results = pd.DataFrame()
results['AAPL'] = cv_aapl,sr_aapl,tr_aapl,sor_aapl
results["TSLA"] = cv_tsla,sr_tsla,tr_tsla,sor_tsla
results.index = ['CV', 'Sharpe_Ratio','Treynor_Ratio','Sortino_Ratio']
results

,AAPL,TSLA
CV,4.628855,7.047703
Sharpe_Ratio,0.197261,0.133054
Treynor_Ratio,0.014741,0.014318
Sortino_Ratio,0.365215,0.288153


# Conclusion
* Python enables us to easily calculate performance metrics of a portfolio.